In [28]:
import numpy as np
import pandas as pd
import torch
import pickle

In [ ]:
def linear_anneal(t, T, start, final, percentage):
    ''' Linear annealing scheduler
    t: current timestep
    T: total timesteps
    start: initial value
    final: value after percentage*T steps
    percentage: percentage of T after which annealing finishes
    '''
    final_from_T = int(percentage * T)
    if t > final_from_T:
        return final
    else:
        return final + (start - final) * (final_from_T - t) / final_from_T

In [5]:
linear_anneal(17, 25, 1.0, 0.01, 0.8)

0.1585

In [6]:
import torch.nn as nn


def build_nn():
    input_size = 72
    output = 8
    layer_sizes = [input_size] + [128, 128, 128] + [output]

    assert len(layer_sizes) > 1
    layers = []
    for index in range(len(layer_sizes) - 1):
        linear = nn.Linear(layer_sizes[index], layer_sizes[index + 1])
        act = nn.ReLU() if index < len(layer_sizes) - 2 else nn.Identity()
        layers += (linear, act)

    return nn.Sequential(*layers)

In [10]:
np.load(
    'C:/Users/evani/OneDrive/AI leiden/Sanquin/RL_matching-main/NN training data/reg_ABDCcEeKkFyaFybJkaJkbMNSs/Q_matrices_0_0.pickle',
    allow_pickle=True)

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [163]:
f = 'C:/Users/evani/OneDrive/AI leiden/Sanquin/RL_matching-main/NN training data/reg_ABDCcEeKkFyaFybJkaJkbMNSs/q_matrices/Q_matrices_0_0.pickle'
s = 'C:/Users/evani/OneDrive/AI leiden/Sanquin/RL_matching-main/NN training data/reg_ABDCcEeKkFyaFybJkaJkbMNSs/states/states_0_0.pickle'

In [164]:
x = torch.load(f, pickle_module=pickle)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [59]:
objects = []
with (open(f, 'rb')) as openfile:
    while True:
        try:
            objects.append((pickle.load(openfile)))
        except EOFError:
            break

In [66]:
np.load(f, allow_pickle=True)

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [70]:
np.load(s, allow_pickle=True)

array([[ 2.,  2., 14., ...,  0.,  0.,  0.],
       [ 2.,  2., 13., ...,  0.,  0.,  0.],
       [ 2.,  2., 13., ...,  0.,  0.,  0.],
       ...,
       [10., 19.,  0., ...,  0.,  0.,  0.],
       [10., 19.,  0., ...,  0.,  0.,  0.],
       [10., 19.,  0., ...,  0.,  0.,  0.]])

In [87]:
layers = [128, 128, 128]
input_size = 72
output = 8

In [158]:
def build_nn():
    input_size = [72]
    output = [8]
    layer_sizes = input_size + [128, 128, 128] + output

    assert len(layer_sizes) > 1
    layers = []
    for index in range(len(layer_sizes) - 1):
        linear = nn.Linear(layer_sizes[index], layer_sizes[index + 1])
        act = nn.ReLU() if index < len(layer_sizes) - 2 else nn.Identity()
        layers += (linear, act)
    return nn.Sequential(*layers)

In [159]:
model = build_nn()

In [160]:
print(model)

Sequential(
  (0): Linear(in_features=72, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=8, bias=True)
  (7): Identity()
)


In [156]:
model(torch.rand(80))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x80 and 72x128)

In [95]:
print(model)

Sequential(
  (0): Linear(in_features=72, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=8, bias=True)
  (7): Identity()
)


In [96]:
[72] + [8] + [128, 128]

[72, 8, 128, 128]

In [94]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print
        name, param.data

In [98]:
pd.DataFrame([500], ['column'])

,0
column,500


In [130]:
import optuna
from optuna.trial import TrialState
import random

In [132]:
def define_model():
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = 3
    layers = []

    in_features = 72
    for i in range(n_layers):
        out_features = random.randint(72, 256)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())

        in_features = out_features
    layers.append(nn.Linear(in_features, 8))

    return nn.Sequential(*layers)

In [134]:
model = define_model()

In [135]:
print(model)

Sequential(
  (0): Linear(in_features=72, out_features=78, bias=True)
  (1): ReLU()
  (2): Linear(in_features=78, out_features=117, bias=True)
  (3): ReLU()
  (4): Linear(in_features=117, out_features=207, bias=True)
  (5): ReLU()
  (6): Linear(in_features=207, out_features=8, bias=True)
)
